# Lab 6 - AWS Bedrock AgentCore COMPLETE Cleanup

## Overview

This notebook provides a comprehensive cleanup solution for ALL AWS resources related to Bedrock AgentCore including:

### Bedrock Resources
- **Bedrock Agents** (with aliases)
- **Memory Stores**
- **Gateways**

### Infrastructure Resources
- **CloudFormation Stacks**
- **S3 Buckets** (with all objects)
- **ECR Repositories** (with all images)
- **CodeBuild Projects**
- **IAM Roles and Policies**
- **SSM Parameters**
- **Secrets Manager Secrets**
- **Cognito Resources**
- **Lambda Functions**
- **CloudWatch Log Groups**

### Important Notes:
- ⚠️ This notebook will DELETE resources - make sure you want to proceed
- ✅ Manual confirmations are required at each step
- ✅ Safe to run multiple times (already-deleted resources are skipped)
- ℹ️ CloudWatch observability metrics may take 5-10 minutes to update after deletion

## Step 1: Import Required Libraries and Initialize Clients

In [ ]:
import boto3
import time
from botocore.exceptions import ClientError
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Initialize AWS clients
bedrock_agent = boto3.client('bedrock-agent')
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')
iam = boto3.client('iam')
ssm = boto3.client('ssm')
secretsmanager = boto3.client('secretsmanager')
cognito_idp = boto3.client('cognito-idp')
s3 = boto3.client('s3')
ecr = boto3.client('ecr')
codebuild = boto3.client('codebuild')
cloudformation = boto3.client('cloudformation')
lambda_client = boto3.client('lambda')
logs = boto3.client('logs')

print("✅ AWS clients initialized successfully")
print(f"📍 Region: {boto3.Session().region_name}")

## Step 2: Configuration

Set the prefixes for resources to be deleted. Modify these if your resources have different names.

In [ ]:
# Configuration - Modify these if your resources have different names
AGENT_PREFIX = "customer_support"
MEMORY_PREFIX = "customer_support"
GATEWAY_PREFIX = "customersupport"
ROLE_NAME = "AmazonBedrockExecutionRoleForAgents_customersupport"
SECRET_NAME = "customer_support_agent"
STACK_PREFIX = "customersupport"  # CloudFormation stack prefix
S3_BUCKET_PREFIX = "customersupport"  # S3 bucket prefix
ECR_REPO_PREFIX = "customersupport"  # ECR repository prefix
CODEBUILD_PREFIX = "customersupport"  # CodeBuild project prefix
LAMBDA_PREFIX = "customersupport"  # Lambda function prefix

print("📋 Configuration:")
print(f"   Agent Prefix: {AGENT_PREFIX}")
print(f"   Memory Prefix: {MEMORY_PREFIX}")
print(f"   Gateway Prefix: {GATEWAY_PREFIX}")
print(f"   IAM Role: {ROLE_NAME}")
print(f"   Secret Name: {SECRET_NAME}")
print(f"   CloudFormation Stack Prefix: {STACK_PREFIX}")
print(f"   S3 Bucket Prefix: {S3_BUCKET_PREFIX}")
print(f"   ECR Repository Prefix: {ECR_REPO_PREFIX}")
print(f"   CodeBuild Prefix: {CODEBUILD_PREFIX}")
print(f"   Lambda Prefix: {LAMBDA_PREFIX}")

## Step 3: Helper Functions

In [ ]:
def print_section(title, emoji="🔧"):
    """Print a formatted section header."""
    print("\n" + "="*60)
    print(f"{emoji} {title}")
    print("="*60)

def print_success(message):
    """Print success message."""
    print(f"✅ {message}")

def print_warning(message):
    """Print warning message."""
    print(f"⚠️  {message}")

def print_info(message):
    """Print info message."""
    print(f"ℹ️  {message}")

def print_error(message):
    """Print error message."""
    print(f"❌ {message}")

def print_deleting(message):
    """Print deletion message."""
    print(f"🗑️  {message}")

def confirm_action(prompt):
    """Ask for user confirmation in Jupyter."""
    response = input(f"{prompt} (yes/no): ").lower().strip()
    return response in ['yes', 'y']

print_success("Helper functions loaded")

## Step 4: Discover Existing Resources

Let's first see what resources currently exist before we start deleting.

In [ ]:
print_section("Discovering Existing Resources", "🔍")

discovered_resources = {
    'agents': [],
    'memories': [],
    'gateways': [],
    'role_exists': False,
    'ssm_params': [],
    'secrets': [],
    'cognito_pools': [],
    'cloudformation_stacks': [],
    's3_buckets': [],
    'ecr_repos': [],
    'codebuild_projects': [],
    'lambda_functions': [],
    'log_groups': []
}

# Check Agents
try:
    response = bedrock_agent.list_agents(maxResults=100)
    agents = response.get('agentSummaries', [])
    discovered_resources['agents'] = [
        agent for agent in agents 
        if AGENT_PREFIX.lower() in agent['agentName'].lower()
    ]
    print(f"📊 Found {len(discovered_resources['agents'])} agent(s)")
    for agent in discovered_resources['agents']:
        print(f"   - {agent['agentName']} (ID: {agent['agentId']}, Status: {agent.get('agentStatus', 'UNKNOWN')})")
except Exception as e:
    print_error(f"Error listing agents: {e}")

# Check Memory Stores
try:
    response = bedrock_agent.list_agent_memories(maxResults=100)
    memories = response.get('agentMemories', [])
    discovered_resources['memories'] = [
        mem for mem in memories 
        if MEMORY_PREFIX.lower() in mem.get('memoryId', '').lower()
    ]
    print(f"📊 Found {len(discovered_resources['memories'])} memory store(s)")
    for mem in discovered_resources['memories']:
        print(f"   - {mem['memoryId']}")
except Exception as e:
    print_error(f"Error listing memories: {e}")

# Check Gateways
try:
    response = bedrock_agent.list_agent_gateways(maxResults=100)
    gateways = response.get('gatewayDetails', [])
    discovered_resources['gateways'] = [
        gw for gw in gateways 
        if GATEWAY_PREFIX.lower() in gw.get('gatewayName', '').lower()
    ]
    print(f"📊 Found {len(discovered_resources['gateways'])} gateway(s)")
    for gw in discovered_resources['gateways']:
        print(f"   - {gw['gatewayName']} (ID: {gw['gatewayId']})")
except Exception as e:
    print_error(f"Error listing gateways: {e}")

# Check CloudFormation Stacks
try:
    response = cloudformation.list_stacks(
        StackStatusFilter=['CREATE_COMPLETE', 'UPDATE_COMPLETE', 'UPDATE_ROLLBACK_COMPLETE']
    )
    stacks = response.get('StackSummaries', [])
    discovered_resources['cloudformation_stacks'] = [
        stack for stack in stacks
        if STACK_PREFIX.lower() in stack['StackName'].lower()
    ]
    print(f"📊 Found {len(discovered_resources['cloudformation_stacks'])} CloudFormation stack(s)")
    for stack in discovered_resources['cloudformation_stacks']:
        print(f"   - {stack['StackName']} (Status: {stack['StackStatus']})")
except Exception as e:
    print_error(f"Error listing CloudFormation stacks: {e}")

# Check S3 Buckets
try:
    response = s3.list_buckets()
    buckets = response.get('Buckets', [])
    discovered_resources['s3_buckets'] = [
        bucket for bucket in buckets
        if S3_BUCKET_PREFIX.lower() in bucket['Name'].lower()
    ]
    print(f"📊 Found {len(discovered_resources['s3_buckets'])} S3 bucket(s)")
    for bucket in discovered_resources['s3_buckets']:
        print(f"   - {bucket['Name']}")
except Exception as e:
    print_error(f"Error listing S3 buckets: {e}")

# Check ECR Repositories
try:
    response = ecr.describe_repositories()
    repos = response.get('repositories', [])
    discovered_resources['ecr_repos'] = [
        repo for repo in repos
        if ECR_REPO_PREFIX.lower() in repo['repositoryName'].lower()
    ]
    print(f"📊 Found {len(discovered_resources['ecr_repos'])} ECR repository(ies)")
    for repo in discovered_resources['ecr_repos']:
        print(f"   - {repo['repositoryName']}")
except Exception as e:
    print_error(f"Error listing ECR repositories: {e}")

# Check CodeBuild Projects
try:
    response = codebuild.list_projects()
    projects = response.get('projects', [])
    discovered_resources['codebuild_projects'] = [
        project for project in projects
        if CODEBUILD_PREFIX.lower() in project.lower()
    ]
    print(f"📊 Found {len(discovered_resources['codebuild_projects'])} CodeBuild project(s)")
    for project in discovered_resources['codebuild_projects']:
        print(f"   - {project}")
except Exception as e:
    print_error(f"Error listing CodeBuild projects: {e}")

# Check Lambda Functions
try:
    response = lambda_client.list_functions()
    functions = response.get('Functions', [])
    discovered_resources['lambda_functions'] = [
        func for func in functions
        if LAMBDA_PREFIX.lower() in func['FunctionName'].lower()
    ]
    print(f"📊 Found {len(discovered_resources['lambda_functions'])} Lambda function(s)")
    for func in discovered_resources['lambda_functions']:
        print(f"   - {func['FunctionName']}")
except Exception as e:
    print_error(f"Error listing Lambda functions: {e}")

# Check IAM Role
try:
    iam.get_role(RoleName=ROLE_NAME)
    discovered_resources['role_exists'] = True
    print(f"📊 Found IAM role: {ROLE_NAME}")
except ClientError as e:
    if e.response['Error']['Code'] != 'NoSuchEntity':
        print_error(f"Error checking role: {e}")

# Check SSM Parameters
try:
    parameter_prefixes = ['/app/customersupport/agentcore/']
    for prefix in parameter_prefixes:
        paginator = ssm.get_paginator('get_parameters_by_path')
        for page in paginator.paginate(Path=prefix, Recursive=True):
            discovered_resources['ssm_params'].extend(page.get('Parameters', []))
    print(f"📊 Found {len(discovered_resources['ssm_params'])} SSM parameter(s)")
    for param in discovered_resources['ssm_params']:
        print(f"   - {param['Name']}")
except Exception as e:
    print_error(f"Error listing SSM parameters: {e}")

# Check Secrets
try:
    secretsmanager.describe_secret(SecretId=SECRET_NAME)
    discovered_resources['secrets'].append(SECRET_NAME)
    print(f"📊 Found secret: {SECRET_NAME}")
except ClientError as e:
    if e.response['Error']['Code'] != 'ResourceNotFoundException':
        print_error(f"Error checking secret: {e}")

# Check Cognito
try:
    response = cognito_idp.list_user_pools(MaxResults=60)
    user_pools = response.get('UserPools', [])
    discovered_resources['cognito_pools'] = [
        pool for pool in user_pools 
        if 'customer' in pool['Name'].lower() or 'support' in pool['Name'].lower()
    ]
    if discovered_resources['cognito_pools']:
        print(f"📊 Found {len(discovered_resources['cognito_pools'])} Cognito pool(s)")
        for pool in discovered_resources['cognito_pools']:
            print(f"   - {pool['Name']} (ID: {pool['Id']})")
except Exception as e:
    print_info("No Cognito resources found")

# Check CloudWatch Log Groups
try:
    response = logs.describe_log_groups(logGroupNamePrefix=f'/aws/lambda/{LAMBDA_PREFIX}')
    discovered_resources['log_groups'] = response.get('logGroups', [])
    print(f"📊 Found {len(discovered_resources['log_groups'])} CloudWatch Log Group(s)")
    for log_group in discovered_resources['log_groups']:
        print(f"   - {log_group['logGroupName']}")
except Exception as e:
    print_error(f"Error listing log groups: {e}")

print_section("Discovery Complete", "✅")
total_resources = (
    len(discovered_resources['agents']) +
    len(discovered_resources['memories']) +
    len(discovered_resources['gateways']) +
    (1 if discovered_resources['role_exists'] else 0) +
    len(discovered_resources['ssm_params']) +
    len(discovered_resources['secrets']) +
    len(discovered_resources['cognito_pools']) +
    len(discovered_resources['cloudformation_stacks']) +
    len(discovered_resources['s3_buckets']) +
    len(discovered_resources['ecr_repos']) +
    len(discovered_resources['codebuild_projects']) +
    len(discovered_resources['lambda_functions']) +
    len(discovered_resources['log_groups'])
)
print(f"📊 Total resources found: {total_resources}")

## Step 5: Delete Bedrock Agents

⚠️ **WARNING**: This will delete all Bedrock agents matching the configured prefix.

The script will:
1. Delete all agent aliases first
2. Then delete the agents themselves
3. Handle agents that are currently in use

In [ ]:
print_section("Deleting Bedrock Agents", "🗑️")

if not discovered_resources['agents']:
    print_info("No agents to delete")
else:
    print(f"Found {len(discovered_resources['agents'])} agent(s) to delete:")
    for agent in discovered_resources['agents']:
        print(f"  - {agent['agentName']} (ID: {agent['agentId']})")
    
    if confirm_action("\n⚠️  Do you want to delete these agents?"):
        for agent in discovered_resources['agents']:
            agent_id = agent['agentId']
            agent_name = agent['agentName']
            
            try:
                print_deleting(f"Deleting agent: {agent_name} ({agent_id})")
                
                # First, list and delete all agent aliases
                try:
                    aliases_response = bedrock_agent.list_agent_aliases(
                        agentId=agent_id,
                        maxResults=100
                    )
                    aliases = aliases_response.get('agentAliasSummaries', [])
                    
                    for alias in aliases:
                        if alias['agentAliasName'] != 'TSTALIASID':  # Don't delete test alias
                            try:
                                bedrock_agent.delete_agent_alias(
                                    agentId=agent_id,
                                    agentAliasId=alias['agentAliasId']
                                )
                                print(f"   ✅ Deleted alias: {alias['agentAliasName']}")
                            except Exception as e:
                                print_warning(f"Error deleting alias {alias['agentAliasName']}: {e}")
                except Exception as e:
                    print_warning(f"Error listing aliases: {e}")
                
                # Delete the agent
                try:
                    bedrock_agent.delete_agent(
                        agentId=agent_id,
                        skipResourceInUseCheck=False
                    )
                    print_success(f"Agent {agent_name} deleted successfully")
                except ClientError as e:
                    if e.response['Error']['Code'] == 'ConflictException':
                        print_warning(f"Agent {agent_name} is in use. Trying forced deletion...")
                        bedrock_agent.delete_agent(
                            agentId=agent_id,
                            skipResourceInUseCheck=True
                        )
                        print_success(f"Agent {agent_name} deleted (forced)")
                    else:
                        raise
                
                time.sleep(2)  # Wait between deletions
                
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print_info(f"Agent {agent_name} already deleted")
                else:
                    print_error(f"Error deleting agent {agent_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting agent {agent_name}: {e}")
        
        print_success("Agent deletion completed")
    else:
        print_error("Agent deletion cancelled by user")

## Step 6: Delete Memory Stores

⚠️ **WARNING**: This will delete all memory stores.

Memory stores contain conversation history and context for agents.

In [ ]:
print_section("Deleting Memory Stores", "🗑️")

if not discovered_resources['memories']:
    print_info("No memory stores to delete")
else:
    print(f"Found {len(discovered_resources['memories'])} memory store(s) to delete:")
    for mem in discovered_resources['memories']:
        print(f"  - {mem['memoryId']}")
    
    if confirm_action("\n⚠️  Do you want to delete these memory stores?"):
        for mem in discovered_resources['memories']:
            memory_id = mem['memoryId']
            
            try:
                print_deleting(f"Deleting memory store: {memory_id}")
                bedrock_agent.delete_agent_memory(memoryId=memory_id)
                print_success(f"Memory store {memory_id} deleted")
                time.sleep(1)
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print_info(f"Memory store {memory_id} already deleted")
                else:
                    print_error(f"Error deleting memory {memory_id}: {e}")
            except Exception as e:
                print_error(f"Error deleting memory {memory_id}: {e}")
        
        print_success("Memory store deletion completed")
    else:
        print_error("Memory store deletion cancelled by user")

## Step 7: Delete Gateways

⚠️ **WARNING**: This will delete all gateways.

Gateways allow agents to connect to external tools and APIs.

In [ ]:
print_section("Deleting Gateways", "🗑️")

if not discovered_resources['gateways']:
    print_info("No gateways to delete")
else:
    print(f"Found {len(discovered_resources['gateways'])} gateway(s) to delete:")
    for gw in discovered_resources['gateways']:
        print(f"  - {gw['gatewayName']} (ID: {gw['gatewayId']})")
    
    if confirm_action("\n⚠️  Do you want to delete these gateways?"):
        for gw in discovered_resources['gateways']:
            gateway_id = gw['gatewayId']
            gateway_name = gw['gatewayName']
            
            try:
                print_deleting(f"Deleting gateway: {gateway_name} ({gateway_id})")
                bedrock_agent.delete_agent_gateway(gatewayId=gateway_id)
                print_success(f"Gateway {gateway_name} deleted")
                time.sleep(1)
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print_info(f"Gateway {gateway_name} already deleted")
                else:
                    print_error(f"Error deleting gateway {gateway_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting gateway {gateway_name}: {e}")
        
        print_success("Gateway deletion completed")
    else:
        print_error("Gateway deletion cancelled by user")

## Step 8: Delete Lambda Functions

⚠️ **WARNING**: This will delete all Lambda functions matching the prefix.

In [ ]:
print_section("Deleting Lambda Functions", "🗑️")

if not discovered_resources['lambda_functions']:
    print_info("No Lambda functions to delete")
else:
    print(f"Found {len(discovered_resources['lambda_functions'])} Lambda function(s) to delete:")
    for func in discovered_resources['lambda_functions']:
        print(f"  - {func['FunctionName']}")
    
    if confirm_action("\n⚠️  Do you want to delete these Lambda functions?"):
        for func in discovered_resources['lambda_functions']:
            func_name = func['FunctionName']
            
            try:
                print_deleting(f"Deleting Lambda function: {func_name}")
                lambda_client.delete_function(FunctionName=func_name)
                print_success(f"Lambda function {func_name} deleted")
                time.sleep(1)
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print_info(f"Lambda function {func_name} already deleted")
                else:
                    print_error(f"Error deleting Lambda function {func_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting Lambda function {func_name}: {e}")
        
        print_success("Lambda function deletion completed")
    else:
        print_error("Lambda function deletion cancelled by user")

## Step 9: Delete CloudWatch Log Groups

⚠️ **WARNING**: This will delete all CloudWatch Log Groups for Lambda functions.

In [ ]:
print_section("Deleting CloudWatch Log Groups", "🗑️")

if not discovered_resources['log_groups']:
    print_info("No log groups to delete")
else:
    print(f"Found {len(discovered_resources['log_groups'])} log group(s) to delete:")
    for log_group in discovered_resources['log_groups']:
        print(f"  - {log_group['logGroupName']}")
    
    if confirm_action("\n⚠️  Do you want to delete these log groups?"):
        for log_group in discovered_resources['log_groups']:
            log_group_name = log_group['logGroupName']
            
            try:
                print_deleting(f"Deleting log group: {log_group_name}")
                logs.delete_log_group(logGroupName=log_group_name)
                print_success(f"Log group {log_group_name} deleted")
                time.sleep(0.5)
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print_info(f"Log group {log_group_name} already deleted")
                else:
                    print_error(f"Error deleting log group {log_group_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting log group {log_group_name}: {e}")
        
        print_success("Log group deletion completed")
    else:
        print_error("Log group deletion cancelled by user")

## Step 10: Delete CodeBuild Projects

⚠️ **WARNING**: This will delete all CodeBuild projects matching the prefix.

In [ ]:
print_section("Deleting CodeBuild Projects", "🗑️")

if not discovered_resources['codebuild_projects']:
    print_info("No CodeBuild projects to delete")
else:
    print(f"Found {len(discovered_resources['codebuild_projects'])} CodeBuild project(s) to delete:")
    for project in discovered_resources['codebuild_projects']:
        print(f"  - {project}")
    
    if confirm_action("\n⚠️  Do you want to delete these CodeBuild projects?"):
        for project in discovered_resources['codebuild_projects']:
            try:
                print_deleting(f"Deleting CodeBuild project: {project}")
                codebuild.delete_project(name=project)
                print_success(f"CodeBuild project {project} deleted")
                time.sleep(1)
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print_info(f"CodeBuild project {project} already deleted")
                else:
                    print_error(f"Error deleting CodeBuild project {project}: {e}")
            except Exception as e:
                print_error(f"Error deleting CodeBuild project {project}: {e}")
        
        print_success("CodeBuild project deletion completed")
    else:
        print_error("CodeBuild project deletion cancelled by user")

## Step 11: Delete ECR Repositories

⚠️ **WARNING**: This will delete all ECR repositories and their images.

All container images in the repositories will be permanently deleted.

In [ ]:
print_section("Deleting ECR Repositories", "🗑️")

if not discovered_resources['ecr_repos']:
    print_info("No ECR repositories to delete")
else:
    print(f"Found {len(discovered_resources['ecr_repos'])} ECR repository(ies) to delete:")
    for repo in discovered_resources['ecr_repos']:
        print(f"  - {repo['repositoryName']}")
    
    if confirm_action("\n⚠️  Do you want to delete these ECR repositories and all their images?"):
        for repo in discovered_resources['ecr_repos']:
            repo_name = repo['repositoryName']
            
            try:
                print_deleting(f"Deleting ECR repository: {repo_name}")
                
                # Force delete (this deletes all images too)
                ecr.delete_repository(
                    repositoryName=repo_name,
                    force=True
                )
                print_success(f"ECR repository {repo_name} deleted (including all images)")
                time.sleep(1)
            except ClientError as e:
                if e.response['Error']['Code'] == 'RepositoryNotFoundException':
                    print_info(f"ECR repository {repo_name} already deleted")
                else:
                    print_error(f"Error deleting ECR repository {repo_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting ECR repository {repo_name}: {e}")
        
        print_success("ECR repository deletion completed")
    else:
        print_error("ECR repository deletion cancelled by user")

## Step 12: Delete S3 Buckets

⚠️ **WARNING**: This will delete all S3 buckets and their contents.

The script will:
1. Delete all objects and versions in the bucket
2. Delete the bucket itself

In [ ]:
print_section("Deleting S3 Buckets", "🗑️")

if not discovered_resources['s3_buckets']:
    print_info("No S3 buckets to delete")
else:
    print(f"Found {len(discovered_resources['s3_buckets'])} S3 bucket(s) to delete:")
    for bucket in discovered_resources['s3_buckets']:
        print(f"  - {bucket['Name']}")
    
    if confirm_action("\n⚠️  Do you want to delete these S3 buckets and ALL their contents?"):
        for bucket in discovered_resources['s3_buckets']:
            bucket_name = bucket['Name']
            
            try:
                print_deleting(f"Deleting S3 bucket: {bucket_name}")
                
                # First, delete all objects and versions
                try:
                    # Check if versioning is enabled
                    versioning = s3.get_bucket_versioning(Bucket=bucket_name)
                    is_versioned = versioning.get('Status') == 'Enabled'
                    
                    if is_versioned:
                        print(f"   🗑️  Deleting all object versions in {bucket_name}...")
                        paginator = s3.get_paginator('list_object_versions')
                        for page in paginator.paginate(Bucket=bucket_name):
                            # Delete versions
                            versions = page.get('Versions', [])
                            if versions:
                                objects_to_delete = [{'Key': v['Key'], 'VersionId': v['VersionId']} for v in versions]
                                s3.delete_objects(
                                    Bucket=bucket_name,
                                    Delete={'Objects': objects_to_delete}
                                )
                            
                            # Delete delete markers
                            delete_markers = page.get('DeleteMarkers', [])
                            if delete_markers:
                                markers_to_delete = [{'Key': m['Key'], 'VersionId': m['VersionId']} for m in delete_markers]
                                s3.delete_objects(
                                    Bucket=bucket_name,
                                    Delete={'Objects': markers_to_delete}
                                )
                    else:
                        print(f"   🗑️  Deleting all objects in {bucket_name}...")
                        paginator = s3.get_paginator('list_objects_v2')
                        for page in paginator.paginate(Bucket=bucket_name):
                            objects = page.get('Contents', [])
                            if objects:
                                objects_to_delete = [{'Key': obj['Key']} for obj in objects]
                                s3.delete_objects(
                                    Bucket=bucket_name,
                                    Delete={'Objects': objects_to_delete}
                                )
                    
                    print(f"   ✅ All objects deleted from {bucket_name}")
                except Exception as e:
                    print_warning(f"Error deleting objects from {bucket_name}: {e}")
                
                # Now delete the bucket
                s3.delete_bucket(Bucket=bucket_name)
                print_success(f"S3 bucket {bucket_name} deleted")
                time.sleep(1)
                
            except ClientError as e:
                if e.response['Error']['Code'] == 'NoSuchBucket':
                    print_info(f"S3 bucket {bucket_name} already deleted")
                else:
                    print_error(f"Error deleting S3 bucket {bucket_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting S3 bucket {bucket_name}: {e}")
        
        print_success("S3 bucket deletion completed")
    else:
        print_error("S3 bucket deletion cancelled by user")

## Step 13: Delete CloudFormation Stacks

⚠️ **WARNING**: This will delete CloudFormation stacks.

CloudFormation will automatically delete associated resources when the stack is deleted.

In [ ]:
print_section("Deleting CloudFormation Stacks", "🗑️")

if not discovered_resources['cloudformation_stacks']:
    print_info("No CloudFormation stacks to delete")
else:
    print(f"Found {len(discovered_resources['cloudformation_stacks'])} CloudFormation stack(s) to delete:")
    for stack in discovered_resources['cloudformation_stacks']:
        print(f"  - {stack['StackName']} (Status: {stack['StackStatus']})")
    
    if confirm_action("\n⚠️  Do you want to delete these CloudFormation stacks?"):
        for stack in discovered_resources['cloudformation_stacks']:
            stack_name = stack['StackName']
            
            try:
                print_deleting(f"Deleting CloudFormation stack: {stack_name}")
                cloudformation.delete_stack(StackName=stack_name)
                print_success(f"CloudFormation stack {stack_name} deletion initiated")
                print_info("Stack deletion is asynchronous. It may take several minutes to complete.")
                time.sleep(2)
            except ClientError as e:
                print_error(f"Error deleting CloudFormation stack {stack_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting CloudFormation stack {stack_name}: {e}")
        
        print_success("CloudFormation stack deletion initiated")
        print_info("\nNote: CloudFormation stacks delete asynchronously.")
        print_info("You can monitor progress in the CloudFormation console.")
    else:
        print_error("CloudFormation stack deletion cancelled by user")

## Step 14: Delete IAM Execution Role

⚠️ **WARNING**: This will delete the IAM role used by Bedrock agents.

The script will:
1. Detach all managed policies
2. Delete all inline policies
3. Delete the role itself

In [ ]:
print_section("Deleting IAM Execution Role", "🗑️")

if not discovered_resources['role_exists']:
    print_info(f"IAM role {ROLE_NAME} not found")
else:
    try:
        print_deleting(f"Deleting IAM role: {ROLE_NAME}")
        
        # Detach all managed policies
        try:
            attached_policies = iam.list_attached_role_policies(RoleName=ROLE_NAME)
            for policy in attached_policies.get('AttachedPolicies', []):
                print(f"   🗑️  Detaching policy: {policy['PolicyName']}")
                iam.detach_role_policy(
                    RoleName=ROLE_NAME,
                    PolicyArn=policy['PolicyArn']
                )
                print(f"   ✅ Policy detached: {policy['PolicyName']}")
        except ClientError as e:
            if e.response['Error']['Code'] != 'NoSuchEntity':
                print_warning(f"Error detaching policies: {e}")
        
        # Delete all inline policies
        try:
            inline_policies = iam.list_role_policies(RoleName=ROLE_NAME)
            for policy_name in inline_policies.get('PolicyNames', []):
                print(f"   🗑️  Deleting inline policy: {policy_name}")
                iam.delete_role_policy(
                    RoleName=ROLE_NAME,
                    PolicyName=policy_name
                )
                print(f"   ✅ Inline policy deleted: {policy_name}")
        except ClientError as e:
            if e.response['Error']['Code'] != 'NoSuchEntity':
                print_warning(f"Error deleting inline policies: {e}")
        
        # Delete the role
        iam.delete_role(RoleName=ROLE_NAME)
        print_success(f"IAM role {ROLE_NAME} deleted")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print_info(f"Role {ROLE_NAME} not found")
        else:
            print_error(f"Error deleting role: {e}")
    except Exception as e:
        print_error(f"Error deleting role: {e}")

## Step 15: Delete SSM Parameters

⚠️ **WARNING**: This will delete all SSM parameters under the configured paths.

Parameters include: runtime ARN, memory ID, gateway ID, client credentials, etc.

In [ ]:
print_section("Deleting SSM Parameters", "🗑️")

if not discovered_resources['ssm_params']:
    print_info("No SSM parameters to delete")
else:
    print(f"Found {len(discovered_resources['ssm_params'])} SSM parameter(s) to delete:")
    for param in discovered_resources['ssm_params']:
        print(f"  - {param['Name']}")
    
    if confirm_action("\n⚠️  Do you want to delete these SSM parameters?"):
        for param in discovered_resources['ssm_params']:
            param_name = param['Name']
            try:
                print_deleting(f"Deleting parameter: {param_name}")
                ssm.delete_parameter(Name=param_name)
                print_success(f"Deleted: {param_name}")
            except ClientError as e:
                if e.response['Error']['Code'] == 'ParameterNotFound':
                    print_info(f"Parameter {param_name} already deleted")
                else:
                    print_error(f"Error deleting {param_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting {param_name}: {e}")
        
        print_success("SSM parameter deletion completed")
    else:
        print_error("SSM parameter deletion cancelled by user")

## Step 16: Delete Secrets Manager Secrets

⚠️ **WARNING**: This will permanently delete secrets from AWS Secrets Manager.

Secrets will be deleted immediately without recovery window.

In [ ]:
print_section("Deleting Secrets Manager Secrets", "🗑️")

if not discovered_resources['secrets']:
    print_info("No secrets to delete")
else:
    print(f"Found secret to delete: {SECRET_NAME}")
    
    if confirm_action("\n⚠️  Do you want to delete this secret?"):
        try:
            print_deleting(f"Deleting secret: {SECRET_NAME}")
            
            # Try to delete without recovery window
            try:
                secretsmanager.delete_secret(
                    SecretId=SECRET_NAME,
                    ForceDeleteWithoutRecovery=True
                )
                print_success(f"Secret {SECRET_NAME} deleted")
            except ClientError as e:
                if e.response['Error']['Code'] == 'InvalidRequestException':
                    # Secret might already be scheduled for deletion
                    print_info("Secret already scheduled for deletion, restoring and force deleting...")
                    try:
                        secretsmanager.restore_secret(SecretId=SECRET_NAME)
                        time.sleep(1)
                        secretsmanager.delete_secret(
                            SecretId=SECRET_NAME,
                            ForceDeleteWithoutRecovery=True
                        )
                        print_success(f"Secret {SECRET_NAME} deleted")
                    except:
                        print_info("Secret will be deleted after recovery window")
                elif e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print_info(f"Secret {SECRET_NAME} not found")
                else:
                    raise
            
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                print_info(f"Secret {SECRET_NAME} not found")
            else:
                print_error(f"Error deleting secret: {e}")
        except Exception as e:
            print_error(f"Error deleting secret: {e}")
    else:
        print_error("Secret deletion cancelled by user")

## Step 17: Delete Cognito Resources (Optional)

⚠️ **WARNING**: This will delete Cognito user pools and associated resources.

Only runs if Cognito resources were found during discovery.

In [ ]:
print_section("Deleting Cognito Resources", "🗑️")

if not discovered_resources['cognito_pools']:
    print_info("No Cognito resources to delete")
else:
    print(f"Found {len(discovered_resources['cognito_pools'])} Cognito user pool(s) to delete:")
    for pool in discovered_resources['cognito_pools']:
        print(f"  - {pool['Name']} (ID: {pool['Id']})")
    
    if confirm_action("\n⚠️  Do you want to delete these Cognito pools?"):
        for pool in discovered_resources['cognito_pools']:
            pool_id = pool['Id']
            pool_name = pool['Name']
            
            try:
                print_deleting(f"Deleting Cognito user pool: {pool_name}")
                
                # First, delete all user pool domains
                try:
                    domain_response = cognito_idp.describe_user_pool(UserPoolId=pool_id)
                    domain = domain_response.get('UserPool', {}).get('Domain')
                    if domain:
                        cognito_idp.delete_user_pool_domain(
                            Domain=domain,
                            UserPoolId=pool_id
                        )
                        print(f"   ✅ Deleted domain: {domain}")
                        time.sleep(2)
                except:
                    pass
                
                # Delete the user pool
                cognito_idp.delete_user_pool(UserPoolId=pool_id)
                print_success(f"Cognito pool {pool_name} deleted")
                time.sleep(1)
                
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print_info(f"Pool {pool_name} already deleted")
                else:
                    print_error(f"Error deleting pool {pool_name}: {e}")
            except Exception as e:
                print_error(f"Error deleting pool {pool_name}: {e}")
        
        print_success("Cognito cleanup completed")
    else:
        print_error("Cognito deletion cancelled by user")

## Step 18: Verification

Let's verify that all resources have been successfully deleted.

In [ ]:
print_section("Verification - Checking for Remaining Resources", "🔍")

time.sleep(3)  # Wait for AWS to propagate changes

remaining = []

# Check agents
try:
    agents = bedrock_agent.list_agents(maxResults=100)
    matching_agents = [
        a for a in agents.get('agentSummaries', [])
        if AGENT_PREFIX.lower() in a['agentName'].lower()
    ]
    if matching_agents:
        remaining.append(f"⚠️  {len(matching_agents)} agent(s) still exist")
        for agent in matching_agents:
            print(f"   - {agent['agentName']} (Status: {agent.get('agentStatus')})")
except:
    pass

# Check memories
try:
    memories = bedrock_agent.list_agent_memories(maxResults=100)
    matching_memories = [
        m for m in memories.get('agentMemories', [])
        if MEMORY_PREFIX.lower() in m.get('memoryId', '').lower()
    ]
    if matching_memories:
        remaining.append(f"⚠️  {len(matching_memories)} memory store(s) still exist")
except:
    pass

# Check gateways
try:
    gateways = bedrock_agent.list_agent_gateways(maxResults=100)
    matching_gateways = [
        g for g in gateways.get('gatewayDetails', [])
        if GATEWAY_PREFIX.lower() in g.get('gatewayName', '').lower()
    ]
    if matching_gateways:
        remaining.append(f"⚠️  {len(matching_gateways)} gateway(s) still exist")
except:
    pass

# Check S3 buckets
try:
    response = s3.list_buckets()
    buckets = response.get('Buckets', [])
    matching_buckets = [
        b for b in buckets
        if S3_BUCKET_PREFIX.lower() in b['Name'].lower()
    ]
    if matching_buckets:
        remaining.append(f"⚠️  {len(matching_buckets)} S3 bucket(s) still exist")
except:
    pass

# Check ECR repos
try:
    response = ecr.describe_repositories()
    repos = response.get('repositories', [])
    matching_repos = [
        r for r in repos
        if ECR_REPO_PREFIX.lower() in r['repositoryName'].lower()
    ]
    if matching_repos:
        remaining.append(f"⚠️  {len(matching_repos)} ECR repository(ies) still exist")
except:
    pass

# Check CodeBuild projects
try:
    response = codebuild.list_projects()
    projects = response.get('projects', [])
    matching_projects = [
        p for p in projects
        if CODEBUILD_PREFIX.lower() in p.lower()
    ]
    if matching_projects:
        remaining.append(f"⚠️  {len(matching_projects)} CodeBuild project(s) still exist")
except:
    pass

# Check CloudFormation stacks
try:
    response = cloudformation.list_stacks(
        StackStatusFilter=['CREATE_COMPLETE', 'UPDATE_COMPLETE', 'DELETE_IN_PROGRESS']
    )
    stacks = response.get('StackSummaries', [])
    matching_stacks = [
        s for s in stacks
        if STACK_PREFIX.lower() in s['StackName'].lower()
    ]
    if matching_stacks:
        remaining.append(f"⚠️  {len(matching_stacks)} CloudFormation stack(s) still exist (may be deleting)")
except:
    pass

# Check IAM role
try:
    iam.get_role(RoleName=ROLE_NAME)
    remaining.append(f"⚠️  IAM role {ROLE_NAME} still exists")
except ClientError:
    pass

if remaining:
    print_warning("Some resources may still exist:")
    for item in remaining:
        print(f"  {item}")
    print("\n" + "="*60)
    print_info("Resources may take a few minutes to fully delete.")
    print_info("CloudFormation stacks can take 5-10 minutes.")
    print_info("Run the verification cell again in 2-3 minutes.")
    print("="*60)
else:
    print_success("All resources have been cleaned up successfully!")
    print("\n" + "="*60)
    print_info("CloudWatch observability metrics may take 5-10 minutes to update.")
    print("="*60)

## Step 19: Summary Report

In [ ]:
print_section("Cleanup Summary Report", "📊")

print("\n📋 Resources Processed:")
print(f"   Bedrock Agents: {len(discovered_resources['agents'])}")
print(f"   Memory Stores: {len(discovered_resources['memories'])}")
print(f"   Gateways: {len(discovered_resources['gateways'])}")
print(f"   Lambda Functions: {len(discovered_resources['lambda_functions'])}")
print(f"   CloudWatch Log Groups: {len(discovered_resources['log_groups'])}")
print(f"   CodeBuild Projects: {len(discovered_resources['codebuild_projects'])}")
print(f"   ECR Repositories: {len(discovered_resources['ecr_repos'])}")
print(f"   S3 Buckets: {len(discovered_resources['s3_buckets'])}")
print(f"   CloudFormation Stacks: {len(discovered_resources['cloudformation_stacks'])}")
print(f"   IAM Role: {'Yes' if discovered_resources['role_exists'] else 'No'}")
print(f"   SSM Parameters: {len(discovered_resources['ssm_params'])}")
print(f"   Secrets: {len(discovered_resources['secrets'])}")
print(f"   Cognito Pools: {len(discovered_resources['cognito_pools'])}")

print("\n" + "="*60)
print("✅ Cleanup process completed")
print("="*60)

print("\n💡 Important Notes:")
print("   1. CloudWatch metrics may take 5-10 minutes to update")
print("   2. CloudFormation stacks delete asynchronously (5-10 minutes)")
print("   3. Some resources have eventual consistency (2-3 minutes)")
print("   4. Re-run verification if resources still appear")
print("   5. Check CloudWatch GenAI Observability for final confirmation")

print("\n" + "="*60)

## Additional Commands (Optional)

### Manual Resource Checks

Use these cells if you need to manually verify or delete specific resources.

In [ ]:
# Manual Agent Listing
print("📋 Listing All Agents:")
try:
    response = bedrock_agent.list_agents(maxResults=100)
    agents = response.get('agentSummaries', [])
    if agents:
        for agent in agents:
            print(f"  - {agent['agentName']} (ID: {agent['agentId']}, Status: {agent.get('agentStatus')})")
    else:
        print("  No agents found")
except Exception as e:
    print(f"  Error: {e}")

In [ ]:
# Manual S3 Bucket Listing
print("📋 Listing All S3 Buckets:")
try:
    response = s3.list_buckets()
    buckets = response.get('Buckets', [])
    if buckets:
        for bucket in buckets:
            print(f"  - {bucket['Name']}")
    else:
        print("  No buckets found")
except Exception as e:
    print(f"  Error: {e}")

In [ ]:
# Manual CloudFormation Stack Status
print("📋 Listing CloudFormation Stack Status:")
try:
    response = cloudformation.list_stacks()
    stacks = response.get('StackSummaries', [])
    if stacks:
        for stack in stacks[:10]:  # Show first 10
            print(f"  - {stack['StackName']}: {stack['StackStatus']}")
    else:
        print("  No stacks found")
except Exception as e:
    print(f"  Error: {e}")

## Troubleshooting Guide

### Common Issues and Solutions

#### 1. Resources Still Showing in CloudWatch
**Issue**: CloudWatch GenAI Observability still shows agents/traces/tokens after deletion

**Solution**: CloudWatch metrics are eventually consistent and can take 5-10 minutes to update. This is normal AWS behavior. Wait and refresh the page.

#### 2. S3 Bucket Deletion Fails
**Issue**: Cannot delete bucket because it's not empty

**Solution**: The notebook automatically empties buckets before deletion. If it fails:
```python
# Manual bucket emptying
bucket_name = 'your-bucket-name'
paginator = s3.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name):
    objects = [{'Key': obj['Key']} for obj in page.get('Contents', [])]
    if objects:
        s3.delete_objects(Bucket=bucket_name, Delete={'Objects': objects})
s3.delete_bucket(Bucket=bucket_name)
```

#### 3. CloudFormation Stack Won't Delete
**Issue**: Stack deletion fails or gets stuck

**Solution**: 
- Check if stack has termination protection enabled
- Some resources may need manual deletion first
- Check CloudFormation Events tab for specific errors

#### 4. ECR Repository Deletion Fails
**Issue**: Repository has images that prevent deletion

**Solution**: The notebook uses `force=True` which deletes all images. If it fails:
```python
# Manual image deletion
repo_name = 'your-repo-name'
images = ecr.list_images(repositoryName=repo_name)['imageIds']
if images:
    ecr.batch_delete_image(repositoryName=repo_name, imageIds=images)
ecr.delete_repository(repositoryName=repo_name)
```

#### 5. CodeBuild Project Still Running
**Issue**: Cannot delete project because builds are in progress

**Solution**: 
```python
# Stop all builds first
project_name = 'your-project-name'
builds = codebuild.list_builds_for_project(projectName=project_name)['ids']
for build_id in builds:
    codebuild.stop_build(id=build_id)
time.sleep(30)
codebuild.delete_project(name=project_name)
```

### AWS Console Verification Checklist

After running this notebook, verify in AWS Console:

1. ✅ **Bedrock Console > Agents**: No customer_support agents
2. ✅ **Bedrock Console > Memory**: No customer_support_agent memory
3. ✅ **Bedrock Console > Gateways**: No customersupport-gw gateway
4. ✅ **S3 Console**: No customersupport buckets
5. ✅ **ECR Console**: No customersupport repositories
6. ✅ **CodeBuild Console**: No customersupport projects
7. ✅ **CloudFormation Console**: No customersupport stacks (or DELETE_COMPLETE)
8. ✅ **Lambda Console**: No customersupport functions
9. ✅ **CloudWatch > Log Groups**: No /aws/lambda/customersupport logs
10. ✅ **CloudWatch > GenAI Observability**: Metrics at 0 (wait 5-10 min)
11. ✅ **IAM > Roles**: No AmazonBedrockExecutionRoleForAgents_customersupport
12. ✅ **Systems Manager > Parameter Store**: No /app/customersupport/ parameters
13. ✅ **Secrets Manager**: No customer_support_agent secret
14. ✅ **Cognito > User Pools**: No customer support related pools

## End of Notebook

---

### Next Steps

1. Verify all resources are deleted in AWS Console
2. Check CloudWatch GenAI Observability dashboard
3. Monitor CloudFormation stack deletion progress
4. If resources persist, wait 5-10 minutes and re-run verification
5. Review CloudTrail logs if you need detailed deletion audit trail

### Important Reminders

- 🔴 This cleanup cannot be undone
- ✅ Notebook is safe to run multiple times
- ⏱️ AWS resources have eventual consistency (2-10 minutes)
- 📊 CloudWatch metrics update on their own schedule
- 🏗️ CloudFormation deletions are asynchronous (5-10 minutes)

---

**Lab 6 Complete** ✅